In [ ]:
import pandas as pd
import numpy as np
import re
import urllib.parse

In [ ]:
df=pd.read_csv('new_data.csv')

In [ ]:
df.columns

In [ ]:
df.shape

In [ ]:
filtered_columns=["Unnamed: 0", "Zip-code", "City", "Type of Property", 
                  "Subtype of Property", "Price", "Construction year", "Bedrooms","Living area",
                  "Kitchen type", "Furnished", "How many fireplaces?", "Terrace","Terrace surface",
                  "Garden","Garden surface","Surface of the plot","Number of frontages",
                  "Swimming pool", "Building condition", "Elevator", "Basement",
                  "Primary energy consumption","CO₂ emission", "Energy class", "Heating type"]
    
df = df[filtered_columns]

In [ ]:
df=df.drop_duplicates()

In [ ]:
df=df.rename(columns = {'Unnamed: 0':'Property ID', 'Zip-code':'Zip code'})

In [ ]:
df=df.rename(columns = {'Price':'Price (€)', 'Living area':'Living area (m²)', 'Terrace surface':'Terrace surface (m²)', 
                        'Garden surface':'Garden surface (m²)', 'Surface of the plot': 'Surface of the plot (m²)', 
                        'Primary energy consumption':'Primary energy consumption (kWh/m²)', 'CO₂ emission':'CO₂ emission (kg CO₂/m²)',
                          })

In [ ]:
df=df.drop_duplicates()

In [ ]:
df['How many fireplaces?']=df['How many fireplaces?'].replace(np.nan,0,regex=True)
df['How many fireplaces?']=df['How many fireplaces?'].replace('Yes',1,regex=True)
df['How many fireplaces?']=df['How many fireplaces?'].replace('No',0,regex=True)
df['Swimming pool']=df['Swimming pool'].replace(np.nan,0,regex=True)
df['Swimming pool']=df['Swimming pool'].replace('Yes',1,regex=True)
df['Swimming pool']=df['Swimming pool'].replace('No',0,regex=True)
df['Elevator']=df['Elevator'].replace(np.nan,0,regex=True)
df['Elevator']=df['Elevator'].replace('Yes',1,regex=True)
df['Elevator']=df['Elevator'].replace('No',0,regex=True)
df['Basement']=df['Basement'].replace(np.nan,0,regex=True)
df['Basement']=df['Basement'].replace('Yes',1,regex=True)
df['Basement']=df['Basement'].replace('No',0,regex=True)
df['Furnished']=df['Furnished'].replace(np.nan,0,regex=True)
df['Furnished']=df['Furnished'].replace('Yes',1,regex=True)
df['Furnished']=df['Furnished'].replace('No',0,regex=True)

In [ ]:
df=df.dropna(subset=['Price (€)', 'Bedrooms', 'Living area (m²)', 'Subtype of Property'])

In [ ]:
df['Price (€)'] = df['Price (€)'].str.split(' ').str[-2]
df['Price (€)'] = pd.to_numeric(df['Price (€)'])
df['Living area (m²)'] = df['Living area (m²)'].str.split(' ').str[0]
df['Garden surface (m²)'] = df['Garden surface (m²)'].str.split(' ').str[0]
df['Terrace surface (m²)'] = df['Terrace surface (m²)'].str.split(' ').str[0]
df['Surface of the plot (m²)'] = df['Surface of the plot (m²)'].str.split(' ').str[0]
df['Primary energy consumption (kWh/m²)'] = df['Primary energy consumption (kWh/m²)'].str.split(' ').str[0]
df['CO₂ emission (kg CO₂/m²)'] = df['CO₂ emission (kg CO₂/m²)'].str.split(' ').str[0]

In [ ]:
filter_G = df["Garden surface (m²)"].isnull()
df.loc[~filter_G,'Garden'] = 'Yes'
df.loc[df['Garden'] == 'Yes', 'Garden'] = 1
df.loc[df["Garden"].isnull(), 'Garden'] = 0
df.loc[df["Garden surface (m²)"].isnull(), 'Garden surface (m²)'] = 0

In [ ]:
filter_G = df["Terrace surface (m²)"].isnull()
df.loc[~filter_G,'Terrace'] = 'Yes'
df.loc[df['Terrace'] == 'Yes', 'Terrace'] = 1
df.loc[df["Terrace"].isnull(), 'Terrace'] = 0
df.loc[df["Terrace surface (m²)"].isnull(), 'Terrace surface (m²)'] = 0

In [ ]:
df['Primary energy consumption (kWh/m²)']=df['Primary energy consumption (kWh/m²)'].replace('Not',np.nan)
df['CO₂ emission (kg CO₂/m²)']=df['CO₂ emission (kg CO₂/m²)'].replace('Not',np.nan)
df['Energy class']=df['Energy class'].replace('Not',np.nan)

In [ ]:
df = df.astype({"Living area (m²)":"float", "Terrace":"float", "Garden":"float",
                "Terrace surface (m²)":"float",
                "Garden surface (m²)":"float","Surface of the plot (m²)":"float",
                "Primary energy consumption (kWh/m²)":"float", 
                "CO₂ emission (kg CO₂/m²)":"float"})

In [ ]:
df['Energy class']=df['Energy class'].replace('Not specified',np.nan)

In [ ]:
kitchen_type_scale={'USA hyper equipped':3, 'USA installed':2, 'USA semi equipped':1, 'USA uninstalled':0,
 'Hyper equipped':3, 'Installed':2, 'Semi equipped':1, 'Not installed':0}

df['Kitchen type scale'] = df['Kitchen type'].map(kitchen_type_scale)

In [ ]:
building_condition_scale={'As new':6,'Just renovated':5, 'Good':4, 'To renovate':2,
 'To restore':1, 'To be done up':3}

df['Building condition scale'] = df['Building condition'].map(building_condition_scale)

In [ ]:
def clean_city_name(city):
    # Remove quotes
    city = city.replace('"', '')
    # Decode URL encoding
    city = urllib.parse.unquote(city)
    return city

df['City'] = df['City'].apply(clean_city_name)

In [ ]:
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [ ]:
df.drop(df[df["Zip code"].str.contains("%20")].index,inplace=True)
df.drop(df[df['Zip code'].str.len() == 5 ].index,inplace=True)
df['Zip code'] = pd.to_numeric(df['Zip code'])
filter_zip = (df['Zip code'] >= 1000) & (df['Zip code'] <= 9999)
df.drop(df.loc[~filter_zip,'Zip code'].index,inplace=True)

In [ ]:
filt_b = (df['Zip code'] >= 1000) & (df['Zip code'] <= 1299)
df.loc[filt_b,'Region'] = 'Brussels capital region'
filt_w = ((df['Zip code'] >= 1300) & (df['Zip code'] <= 1499)) | ((df['Zip code'] >=4000) & (df['Zip code'] <=7999))
df.loc[filt_w,'Region'] = 'Walloon'
filt_f = ((df['Zip code'] >= 1500) & (df['Zip code'] <= 3999)) | ((df['Zip code'] >=8000) & (df['Zip code'] <=9999))
df.loc[filt_f,'Region'] = 'Flemish'

In [ ]:
df = df.drop(df.loc[df['Price (€)'] == 35000000].index)
df = df.drop(df.loc[df['Living area (m²)'] == 1.0].index)

In [ ]:
df['Price per m²'] = df['Price (€)']/df['Living area (m²)']
df['Price per m²'].round()

In [ ]:
df=df.drop_duplicates()

In [ ]:
#df.to_csv("updated_cleaned_data.csv", index=False)

In [ ]:
# # Import necessary modules
# from sklearn.linear_model import LinearRegression
# from sklearn.metrics import mean_squared_error
# from sklearn.model_selection import train_test_split

# # Create training and test sets
# X_train, X_test, y_train, y_test = train_test_split( X, y, test_size = 0.3, random_state=42)

# # Create the regressor: reg_all
# reg_all = LinearRegression()

# # Fit the regressor to the training data
# reg_all.fit(X_train,y_train)

# # Predict on the test data: y_pred
# y_pred = reg_all.predict(X_test)

# # Compute and print R^2 and RMSE
# print("R^2: {}".format(reg_all.score(X_test, y_test)))
# rmse = np.sqrt(mean_squared_error(y_test, y_pred))
# print("Root Mean Squared Error: {}".format(rmse))


In [5]:
import pandas as pd
df = pd.read_csv('updated_cleaned_data.csv')
df.describe().round()


,Property ID,Zip code,Price (€),Construction year,Bedrooms,Living area (m²),Furnished,How many fireplaces?,Terrace,Terrace surface (m²),...,Surface of the plot (m²),Number of frontages,Swimming pool,Elevator,Basement,Primary energy consumption (kWh/m²),CO₂ emission (kg CO₂/m²),Kitchen type scale,Building condition scale,Price per m²
count,15146.0,15146.0,15146.0,9869.0,15146.0,15146.0,15146.0,15146.0,15146.0,15146.0,...,9278.0,11826.0,15146.0,15146.0,15146.0,12316.0,3533.0,10515.0,11537.0,15146.0
mean,10481706.0,4730.0,574772.0,1974.0,3.0,217.0,0.0,0.0,1.0,13.0,...,2731.0,3.0,0.0,0.0,0.0,365.0,964.0,2.0,4.0,2852.0
std,200625.0,3069.0,629446.0,41.0,2.0,208.0,0.0,0.0,0.0,33.0,...,29948.0,1.0,0.0,0.0,0.0,1686.0,4797.0,1.0,1.0,1953.0
min,4611027.0,1000.0,19000.0,1753.0,1.0,14.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,-10.0,-4.0,0.0,1.0,32.0
25%,10470283.0,1950.0,273611.0,1956.0,2.0,114.0,0.0,0.0,0.0,0.0,...,260.0,2.0,0.0,0.0,0.0,176.0,41.0,2.0,4.0,1839.0
50%,10561596.0,3970.0,395000.0,1978.0,3.0,170.0,0.0,0.0,1.0,0.0,...,659.0,3.0,0.0,0.0,0.0,272.0,67.0,2.0,4.0,2486.0
75%,10573953.0,8300.0,635000.0,2007.0,4.0,254.0,0.0,0.0,1.0,16.0,...,1448.0,4.0,0.0,0.0,1.0,406.0,111.0,3.0,6.0,3290.0
max,10584959.0,9992.0,11000000.0,2033.0,32.0,11000.0,1.0,6.0,1.0,1500.0,...,2400000.0,14.0,1.0,1.0,1.0,85624.0,88028.0,3.0,6.0,58824.0


In [6]:
df.isnull().sum()

Property ID                                0
Zip code                                   0
City                                       0
Type of Property                           0
Subtype of Property                        0
Price (€)                                  0
Construction year                       5277
Bedrooms                                   0
Living area (m²)                           0
Kitchen type                            4631
Furnished                                  0
How many fireplaces?                       0
Terrace                                    0
Terrace surface (m²)                       0
Garden                                     0
Garden surface (m²)                        0
Surface of the plot (m²)                5868
Number of frontages                     3320
Swimming pool                              0
Building condition                      3609
Elevator                                   0
Basement                                   0
Primary en

In [14]:
df.loc[df['Swimming pool'] == 0,'Swimming pool'].count()

14416

In [15]:
df['Swimming pool'].count()

15146

In [16]:
ratio = df.loc[df['Swimming pool'] == 0,'Swimming pool'].count() * 100/ df['Swimming pool'].count()
ratio

95.18024560940182

In [ ]:
# def subset_by_iqr(df, column, whisker_width=1.5):
#     """Remove outliers from a dataframe by column, including optional 
#        whiskers, removing rows for which the column value are 
#        less than Q1-1.5IQR or greater than Q3+1.5IQR.
#     Args:
#         df (`:obj:pd.DataFrame`): A pandas dataframe to subset
#         column (str): Name of the column to calculate the subset from.
#         whisker_width (float): Optional, loosen the IQR filter by a
#                                factor of `whisker_width` * IQR.
#     Returns:
#         (`:obj:pd.DataFrame`): Filtered dataframe
#     """
#     # Calculate Q1, Q2 and IQR
#     q1 = df[column].quantile(0.25)                 
#     q3 = df[column].quantile(0.75)
#     iqr = q3 - q1
#     # Apply filter with respect to IQR, including optional whiskers
#     filter = (df[column] >= q1 - whisker_width*iqr) & (df[column] <= q3 + whisker_width*iqr)
#     return df.loc[filter]                                                     

# # Example for whiskers = 1.5, as requested by the OP
# df_filtered = subset_by_iqr(df, 'column_name', whisker_width=1.5)

In [ ]:
# Import necessary modules
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Create training and test sets
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size = 0.3, random_state=42)

# Create the regressor: reg_all
reg_all = LinearRegression()

# Fit the regressor to the training data
reg_all.fit(X_train,y_train)

# Predict on the test data: y_pred
y_pred = reg_all.predict(X_test)

# Compute and print R^2 and RMSE
print("R^2: {}".format(reg_all.score(X_test, y_test)))
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error: {}".format(rmse))

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() # 1
X_train_scaled = scaler.fit_transform(X_train) # 2
model = LinearRegression()#3
model = model.fit(X_train_scaled, y_train)#4
X_test_scaled = scaler.transform(X_test)#5
y_pred = model.predict(X_test_scaled)#6

# Calculate MSE (Lower better)
mse = mean_squared_error(y_test, y_pred)
print("MSE of testing set:", mse)



# Then fit the scaling on the training set
# Convert Feature/Column with Scaler
scaler = MinMaxScaler()
# Note: the columns have already been selected
X_train_scaled = scaler.fit_transform(X_train)

# Calling LinearRegression
model = LinearRegression()

# Fit linearregression into training data
model = model.fit(X_train_scaled, y_train)

# Now we need to scale the test set features
X_test_scaled = scaler.transform(X_test)
y_pred = model.predict(X_test_scaled)
# y has not been scaled so nothing else to do 

# Calculate MSE (Lower better)
mse = mean_squared_error(y_test, y_pred)
print("MSE of testing set:", mse)